# 🧩 Transducible Functions Tutorial

**Transducible functions** are Python functions that can be *lifted* into **Agentic Structures (AGs)** — 
structured, traceable computational units used in the `agentics` framework.

They extend ordinary Python functions with GenAI built in capabilities by:
- Capturing structured **input/output models**
- Supporting **async execution**
- Integrating with LLMs using **Agentics Transductional Engine**
- Supporting **Parallel mapping (amap)** operations

In this tutorial, you’ll learn how to:
1. Define and decorate transducible functions  
2. Execute them both singly and in parallel
3. Use the framework to enable Map Reduce


### Defining Transducible Functions

Agentics allows you to wrap any Python function and turn it into a transducible function, giving it native LLM-powered transformation capabilities.


#### Defining Source and Target Types

In [ ]:
from typing import Optional
from pydantic import BaseModel
from agentics.core.transducible_functions import transducible, Transduce

class GenericInput(BaseModel):
    content: Optional[str] = None

class Email(BaseModel):
    to: Optional[str] = None
    subject:Optional[str]=None
    body: Optional[str]=None

#### Define Transducible Functions

In [ ]:

@transducible()
async def write_an_email(state: GenericInput) -> Email:
    """Write an email about the provided content. Elaborate on that and make up content as needed"""
    # example code to modify states before transduction
    return Transduce(state)

## Transducible functions can be introspected to easily get their input , output , description and original function
print(write_an_email.input_model)
print(write_an_email.target_model)
print(write_an_email.description)
print(write_an_email.__original_fn__)


single_mail = await write_an_email(GenericInput(content=f"I have made great progress with agentics"))
print(single_mail.model_dump_json(indent=2))



### Hybrid (Code and llm) functions

A transducible function must follow two rules:

1. It must accept exactly one input parameter : The parameter must be an instance of a Pydantic SOURCE type.
This enforces strong typing and guarantees predictable I/O behavior.

2. It must return one of the following:
- A TARGET Pydantic object: If the function directly returns an instance of the TARGET type, no LLM call is made — the function behaves as a pure Python transformation.
- B. Transduce(obj) :If it returns Transduce(obj) (where obj is a SOURCE-type Pydantic object), this triggers an LLM-based transduction, meaning: the input object is serialized, instructions are applied, the LLM produces an output object of the TARGET type via structured decoding.

This mechanism turns a simple Python function into a fully controllable, typed, LLM-driven transformation.

In [ ]:

@transducible()
async def write_an_email_code_only(state: GenericInput) -> Email:    
    return Email(body= state.content)

@transducible()
async def write_an_email_to_lisa(state: GenericInput) -> Email:
    """Write an email about the provided content. Elaborate on that and make up content as needed"""
    # example code to modify states before transduction
    state.content=state.content + " send it to Lisa"
    return Transduce(state)

#code_only_mail = await write_an_email_code_only(GenericInput(content=f"I have made great progress with agentics"))
hybrid_code_llm_mail = await write_an_email_to_lisa(GenericInput(content=f"I have made great progress with agentics"))

#print(code_only_mail.model_dump_json(indent=2))
print(hybrid_code_llm_mail.model_dump_json(indent=2))

### Using tools

transducible functions can take a list of tools as an argument when they are initialized . 
You can use both CrewAI tools and MCP tools seamlessy. 

In [ ]:
from crewai.tools import tool
from ddgs import DDGS
@tool("web_search")
def web_search(query: str) -> str:
    """return spippets of text extracted from duck duck go search for the given
        query :  using DDGS search operators
        
    DDGS search operators Guidelines in the table below:
    Query example	Result
    cats dogs	Results about cats or dogs
    "cats and dogs"	Results for exact term "cats and dogs". If no results are found, related results are shown.
    cats -dogs	Fewer dogs in results
    cats +dogs	More dogs in results
    cats filetype:pdf	PDFs about cats. Supported file types: pdf, doc(x), xls(x), ppt(x), html
    dogs site:example.com	Pages about dogs from example.com
    cats -site:example.com	Pages about cats, excluding example.com
    intitle:dogs	Page title includes the word "dogs"
    inurl:cats	Page url includes the word "cats"""
    return str(DDGS().text(query, max_results=20))


When using tools , you can set reasoning=True to use the planning strategy implemented by crewAI. Setting verbose_agent=True print out agent logs, max_iter is the maximun number of steps (mostly tool calls) allowed before executing a single transduction

In [ ]:
class WebSearchResult(BaseModel):
    report_summary:Optional[str]=None
    relevant_sources:Optional[list[str]]=None


@transducible(tools=[web_search], reasoning=True, verbose_agent=True, max_iter=10)
async def answer_question_after_lookup(query: GenericInput) -> WebSearchResult:
    "perform an extensive web search to provide an answer to the input question with supporting evidence. Use your tool to look it up" 
    return Transduce(query)

out = await answer_question_after_lookup(GenericInput(content="who was the NYC mayor in 2025 and in 1998?"))
print(out.model_dump_json(indent=2))

## Map Reduce Framework


🟩 aMap

aMap allows you to apply a transducible function asynchronously over a list of SOURCE-type objects.
Each element is processed in parallel by the Agentics execution engine, and the final result is returned as a list of TARGET-type Pydantic objects.

In other words:
	•	You pass a list of SOURCE instances
	•	Agentics schedules each transduction asynchronously
	•	The function returns a list of TARGET instances
	•	Both pure-Python logic and Transduce(...)-triggered LLM calls are supported

This enables highly scalable workloads, such as batch inference, dataset scanning, or parallel evidence extraction in Map–Reduce pipelines.

In [ ]:

multiple_mails = await write_an_email([GenericInput(content=f"I have made great progress with agentics"),
                                        GenericInput(content=f"I have made great progress with LTA")])
for mail in multiple_mails:
    print(mail.model_dump_json(indent=2))


🟦 AReduce


A reduce function aggregates multiple SOURCE_TYPE instances into a single TARGET_TYPE instance.
When a reduce function returns Transduce(source), it triggers an Agentics areduce operation.

How it works
	•	The full list of inputs is automatically batched into chunks of ~10k tokens.
	•	Each batch is sent to the LLM asynchronously for transduction.
	•	The LLM produces one TARGET_TYPE object per batch.
	•	The final result returned is a list of TARGET_TYPE instances, one for each batch.

This mechanism enables scalable reduction over large structured datasets—ideal for tasks such as evidence aggregation, dataset summarization, statistical description, or multi-row reasoning.



In [ ]:
class Number(BaseModel):
    number:int

@transducible(areduce=True)
async def sum_all_numbers_code(numbers: list[Number]) -> Number :
    sum=0
    for number in numbers:
        sum+=number.number
    return Number(number=sum)


@transducible(areduce=True)
async def sum_all_numbers_llm(numbers: list[Number]) -> Number :
    return Transduce(numbers)


print(await sum_all_numbers_code([Number(number=1),Number(number=3)]))
print(await sum_all_numbers_llm([Number(number=1),Number(number=3)]))